# Idea
Do some training

In [52]:
SEED = 133
TIME_WINDOW = "30s"
DATA_FILE = '../../data/all_df_30s_Xi_sensor_1.csv'

COLUMNS_SELECTION_FILE_FROM_RP4 = "../other_time_windows/weight_selected_30s.csv"

In [53]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

assert Path(DATA_FILE).exists()

In [54]:
import sys
repo_base_path = Path("./../../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))
sys.path.append("..")

# from py_dataset import get_all_files_df
# from py_dataset import read_in_files
from py_dataset import feature_plotting
# from py_dataset import feature_selection
# from py_dataset import net_feature_engineering
# from py_dataset import entropy_feature_engineering
# from py_dataset.classes import DataSource
from py_dataset import sys_func

from py_dataset.classes import DataSource



In [55]:
LOG_PATH = (repo_base_path / "training" / "official_logs")
LOG_NAME = f"single-layer_300_{TIME_WINDOW}_Xi_sensor_1_rp3"
LOG_NAME_EXPERIMENT_2 = f"single-layer_30_from_rp4_weights_{TIME_WINDOW}_Xi_sensor_1_rp3"
LOG_NAME_EXPERIMENT_3 = f"multi-layer_30_from_rp4_weights_{TIME_WINDOW}_Xi_sensor_1_rp3"

assert LOG_PATH.exists()
assert not (LOG_PATH / LOG_NAME).exists(), "are you sure to run this again?"
assert not (LOG_PATH / LOG_NAME_EXPERIMENT_2).exists(), "are you sure to run this again?"
assert not (LOG_PATH / LOG_NAME_EXPERIMENT_3).exists(), "are you sure to run this again?"

AssertionError: are you sure to run this again?

In [ ]:
df = pd.read_csv(DATA_FILE)
df.set_index("timestamp", inplace=True)

/tmp/ipykernel_239875/1931635172.py:1: DtypeWarning: Columns (189) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_FILE)


In [ ]:
assert df["label"].unique().shape == (10,)

In [ ]:
selected_columns = pd.read_csv(COLUMNS_SELECTION_FILE_FROM_RP4)["0"].values

### Preprocessings
- remove Null values
- drop timestamp
- labelEncode
- normalize

In [ ]:
print(df.shape)
old_df = df.copy()
df.dropna(axis=0, how='any', inplace=True)

print(df.shape)

(38686, 288)
(21550, 288)


In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
label_encoder = sys_func.create_label_encoder_for_labels()
df['label'] = label_encoder.transform(df['label'])

In [ ]:
df = feature_plotting.test_transform_all_to_numeric_columns(df)

In [ ]:
cols_to_exclude_from_scaling = ["label"]
vectors = feature_plotting.minmax_scale_features(df,remove_outliers_iqr_all_columns=False,
                                                        cols_to_exclude_from_scaling=cols_to_exclude_from_scaling)
# print(vectors.shape)
# print(" with dropped null: ", vectors.dropna(axis=0, how='any', inplace=False).shape)

In [ ]:
df["label"].unique(), vectors["label"].unique()

(array([0, 8, 4, 1, 5, 2, 7, 6, 3]), array([0, 8, 4, 1, 5, 2, 7, 6, 3]))

### Setup

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomeDataset(Dataset):
    def __init__(self, some_df:pd.DataFrame):
        
        self.x = torch.tensor(some_df.drop(columns=["label"], inplace=False).values, dtype=torch.float32)
        self.labels = torch.tensor(some_df["label"].values, dtype=torch.uint8)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # print(self.x[idx], self.labels[idx])
        return self.x[idx], self.labels[idx]

In [ ]:
from mlp_single_layer import MalwaresModelLinearLayer
from torch.utils.data import DataLoader
import lightning
from sklearn.model_selection import train_test_split
from lightning import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

tensorboard = pl_loggers.TensorBoardLogger(save_dir=LOG_PATH, name=LOG_NAME)

# split into train and val
train, val = train_test_split(vectors, test_size=0.2, random_state=SEED)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True, logger=tensorboard) #logger=None, 


model = MalwaresModelLinearLayer(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_), seed=SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 2.6 K 
------------------------------------------------------------
2.6 K     Trainable params
0         Non-trainable params
2.6 K     Total params
0.010     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0,   0,   0,   1, 105,   0,   0,   0],
        [  0,   0,   0,   0,   5, 102,   0,   0,   0],
        [  0,   0,   0,   0,   0,  93,   0,   0,   0],
        [ 11,   0,   0,   0,   0, 106,   1,   0,   0],
        [  0,   0,   0,   0,   0,  98,   0,   0,   0],
        [  0,   0,   0,   0,   0, 115,   0,   0,   0],
        [ 15,   0,   0,   0,  12, 100,   0,   0,   0],
        [  9,   0,   0,   0,   2,  99,   0,   0,   0],
        [  0,  12,   0,   0,   0, 114,   0,   0,   0]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 14
ValidationEpoch/CM
 tensor([[32902,     0,   221,     0,   419,   153,     0,     0,    11],
        [    0, 33611,     0,     0,     8,   158,     0,     0,     0],
        [  865,     3, 32553,     0,   112,   230,     0,     0,     0],
        [   15,     8,    18, 33391,     0,   159,    57,     0,     0],
        [  159,     0,   129,     0, 33251,   159,     0,     0,     0],
        [    3,    45,   126,     0,     0, 33541,     0,     0,     0],
        [   23,     0,     0,     0,    14,   143, 32703,   214,     0],
        [   11,     0,     0,     0,     4,   149,   282, 33264,     0],
        [    0,    55,    55,     0,     0,   167,     0,     0, 33309]])
Epoch end: Train, epoch number: 13
TrainEpoch/CM
 tensor([[32902,     0,   221,     0,   419,   153,     0,     0,    11],
        [    0, 33611,     0,     0,     8,   158,     0,     0,     0],
        [  865,     3, 32553,     0,   112,   230,     0,     0,     0],
        [   1

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 15
ValidationEpoch/CM
 tensor([[35302,     0,   221,     0,   419,   153,     0,     0,    11],
        [    0, 36016,     0,     0,     8,   158,     0,     0,     0],
        [  875,     3, 34948,     0,   112,   230,     0,     0,     0],
        [   15,     8,    18, 35786,     0,   159,    57,     0,     0],
        [  159,     0,   129,     0, 35651,   159,     0,     0,     0],
        [    3,    45,   126,     0,     0, 35941,     0,     0,     0],
        [   23,     0,     0,     0,    14,   143, 35058,   214,     0],
        [   11,     0,     0,     0,     4,   149,   282, 35664,     0],
        [    0,    55,    55,     0,     0,   167,     0,     0, 35699]])
Epoch end: Train, epoch number: 14
TrainEpoch/CM
 tensor([[35302,     0,   221,     0,   419,   153,     0,     0,    11],
        [    0, 36016,     0,     0,     8,   158,     0,     0,     0],
        [  875,     3, 34948,     0,   112,   230,     0,     0,     0],
        [   1

Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 1.000. Signaling Trainer to stop.


In [ ]:
model.eval()

MalwaresModelLinearLayer(
  (train_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Train/
  )
  (val_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Validation/
  )
  (test_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Test/
  )
  (cm): MulticlassConfusionMatrix()
  (criterion): CrossEntropyLoss()
  (l1): Linear(in_features=287, out_features=9, bias=True)
)

In [ ]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelLinearLayer.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/official_logs/single-layer_300_30s_Xi_sensor_1_rp3/version_0/checkpoints/epoch=9-step=350.ckpt
tensor(1., device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[480,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0, 474,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 479,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0, 462,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 485,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 478,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0, 482,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 485,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 485]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy                 1.0
      Test/F1Score                  1.0
        Test/Loss          0.024981120601296425
     Test/Precision                 1.0
   

[{'Test/Loss': 0.024981120601296425,
  'Test/Accuracy': 1.0,
  'Test/Precision': 1.0,
  'Test/Recall': 1.0,
  'Test/F1Score': 1.0,
  'TestEpoch/Accuracy': 1.0,
  'TestEpoch/Precision': 1.0,
  'TestEpoch/Recall': 1.0,
  'TestEpoch/F1Score': 1.0}]

### Train again

In [ ]:
print ("going to remove columns: ", set(selected_columns) - set(vectors.columns))
selected_columns_that_are_in_rp3_dataset = list(set(selected_columns) & set(vectors.columns))

going to remove columns:  {'armv7_cortex_a15/br_mis_pred/'}


In [56]:
vectors[selected_columns_that_are_in_rp3_dataset].shape

(21550, 32)

In [57]:
subset = vectors[selected_columns_that_are_in_rp3_dataset]
subset["label"] = df["label"]

/tmp/ipykernel_239875/468855836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["label"] = df["label"]


In [58]:
# split into train and val
train, val = train_test_split(subset, test_size=0.2, random_state=SEED)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

tensorboard = pl_loggers.TensorBoardLogger(save_dir=LOG_PATH, name=LOG_NAME_EXPERIMENT_2)
trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True, logger=tensorboard) #logger=None, 


model = MalwaresModelLinearLayer(input_size = subset.shape[1]-1, output_size = len(label_encoder.classes_), seed=SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [59]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 288   
------------------------------------------------------------
288       Trainable params
0         Non-trainable params
288       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0, 106,   0,   0,   0,   0,   0,   0],
        [ 27,   0,  80,   0,   0,   0,   0,   0,   0],
        [  0,   0,  82,   0,  11,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 118],
        [  0,   0,  70,   0,   0,   0,  27,   1,   0],
        [  0,   0,  98,   0,  17,   0,   0,   0,   0],
        [  5,   0,  13,   0,  56,   0,   0,   0,  53],
        [  1,   0,  30,   0,  32,   0,   0,   0,  47],
        [  0,   0, 125,   0,   1,   0,   0,   0,   0]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Epoch end: Validation, epoch number: 18
ValidationEpoch/CM
 tensor([[38353,     0,  1863,     0,  1020,    19,     0,     0,  2051],
        [   76, 42912,   230,     0,     8,   170,     1,     0,     0],
        [ 4533,     0, 36732,     0,  1686,    79,     0,     0,   353],
        [    0,     2,     1, 39786,     0,     0,  2601,    15,   823],
        [   35,     0,   584,     0, 42509,    18,    45,     2,   105],
        [    0,     3,   185,     0,   473, 42654,     0,     0,     0],
        [   10,    14,    22,     0,   176,     0, 32364,  9753,   178],
        [    3,    85,    54,     0,    95,     1,  8554, 34345,   173],
        [ 1629,     0,   730,     0,   324,   110,     1,     0, 40352]])
Epoch end: Train, epoch number: 17
TrainEpoch/CM
 tensor([[38353,     0,  1863,     0,  1020,    19,     0,     0,  2051],
        [   76, 42912,   230,     0,     8,   170,     1,     0,     0],
        [ 4533,     0, 36732,     0,  1686,    79,     0,     0,   353],
        [    

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 19
ValidationEpoch/CM
 tensor([[40636,     0,  1873,     0,  1020,    19,     0,     0,  2158],
        [   76, 45317,   230,     0,     8,   170,     1,     0,     0],
        [ 4641,     0, 39019,     0,  1686,    79,     0,     0,   363],
        [    0,     2,     1, 42181,     0,     0,  2601,    15,   823],
        [   35,     0,   584,     0, 44909,    18,    45,     2,   105],
        [    0,     3,   185,     0,   473, 45054,     0,     0,     0],
        [   10,    14,    22,     0,   176,     0, 34380, 10092,   178],
        [    3,    85,    54,     0,    95,     1,  9005, 36294,   173],
        [ 1714,     0,   730,     0,   324,   110,     1,     0, 42657]])


Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.947. Signaling Trainer to stop.


Epoch end: Train, epoch number: 18
TrainEpoch/CM
 tensor([[40636,     0,  1873,     0,  1020,    19,     0,     0,  2158],
        [   76, 45317,   230,     0,     8,   170,     1,     0,     0],
        [ 4641,     0, 39019,     0,  1686,    79,     0,     0,   363],
        [    0,     2,     1, 42181,     0,     0,  2601,    15,   823],
        [   35,     0,   584,     0, 44909,    18,    45,     2,   105],
        [    0,     3,   185,     0,   473, 45054,     0,     0,     0],
        [   10,    14,    22,     0,   176,     0, 34380, 10092,   178],
        [    3,    85,    54,     0,    95,     1,  9005, 36294,   173],
        [ 1714,     0,   730,     0,   324,   110,     1,     0, 42657]])


In [60]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelLinearLayer.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = subset.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/official_logs/single-layer_30_from_rp4_weights_30s_Xi_sensor_1_rp3/version_0/checkpoints/epoch=13-step=490.ckpt
tensor(0.9466, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[452,   0,   4,   0,   0,   0,   0,   0,  24],
        [  0, 474,   0,   0,   0,   0,   0,   0,   0],
        [ 18,   0, 460,   0,   0,   0,   0,   0,   1],
        [  0,   0,   0, 462,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 485,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 478,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0, 415,  67,   0],
        [  0,   0,   0,   0,   0,   0,  97, 388,   0],
        [ 22,   0,   0,   0,   0,   0,   0,   0, 463]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9466128349304199
      Test/F1Score          0.9464346170425415
        Test/Loss            0.304229736328125
     Test/Precision         0.

[{'Test/Loss': 0.304229736328125,
  'Test/Accuracy': 0.9466128349304199,
  'Test/Precision': 0.9474481344223022,
  'Test/Recall': 0.9466128349304199,
  'Test/F1Score': 0.9464346170425415,
  'TestEpoch/Accuracy': 0.946403980255127,
  'TestEpoch/Precision': 0.9469047784805298,
  'TestEpoch/Recall': 0.946403980255127,
  'TestEpoch/F1Score': 0.9464320540428162}]

In [61]:
sys.path.append(str(repo_base_path  / "training"))

In [62]:
# tensorboard = pl_loggers.TensorBoardLogger('./')

from mlp import MalwaresModelMLP

# split into train and val
train, val = train_test_split(subset, test_size=0.2, random_state=SEED)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

tensorboard = pl_loggers.TensorBoardLogger(save_dir=LOG_PATH, name=LOG_NAME_EXPERIMENT_3)
trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True, logger=tensorboard) #logger=None, 


model = MalwaresModelMLP(input_size = subset.shape[1]-1, output_size = len(label_encoder.classes_), seed=SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [63]:

trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 960   
6 | l2            | Linear                    | 930   
7 | l3            | Linear                    | 279   
------------------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0, 106,   0,   0,   0,   0,   0,   0],
        [  0,   0,  96,   0,   0,   0,  11,   0,   0],
        [  0,   0,  91,   0,   0,   0,   2,   0,   0],
        [  0,   0, 118,   0,   0,   0,   0,   0,   0],
        [  0,   0,  98,   0,   0,   0,   0,   0,   0],
        [  0,   0, 106,   0,   0,   0,   9,   0,   0],
        [  0,   0,  80,   0,   0,   0,  47,   0,   0],
        [  0,   0,  92,   0,   0,   0,  18,   0,   0],
        [  0,   0, 126,   0,   0,   0,   0,   0,   0]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (35) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 17
ValidationEpoch/CM
 tensor([[37476,     0,  1112,     0,  1301,     0,     0,     0,  1017],
        [    0, 40709,   210,     0,     0,     0,    73,     0,     0],
        [  477,     0, 38876,     0,  1425,    12,     4,     0,   184],
        [    0,     0,   340, 39554,     0,     0,   426,   513,     0],
        [   43,     0,   738,     0, 40115,     0,     0,     0,     2],
        [    0,   178,   270,     0,   322, 40031,   110,     0,     4],
        [    0,     0,   114,     0,     0,     0, 35261,  4787,     0],
        [    0,     0,   135,     0,     0,     0,  3792, 36983,     0],
        [  771,     0,   411,     0,   734,    61,     0,     0, 38779]])
Epoch end: Train, epoch number: 16
TrainEpoch/CM
 tensor([[37476,     0,  1112,     0,  1301,     0,     0,     0,  1017],
        [    0, 40709,   210,     0,     0,     0,    73,     0,     0],
        [  477,     0, 38876,     0,  1425,    12,     4,     0,   184],
        [    

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 18
ValidationEpoch/CM
 tensor([[39834,     0,  1120,     0,  1301,     0,     0,     0,  1051],
        [    0, 43114,   210,     0,     0,     0,    73,     0,     0],
        [  487,     0, 41271,     0,  1425,    12,     4,     0,   184],
        [    0,     0,   340, 41949,     0,     0,   426,   513,     0],
        [   43,     0,   738,     0, 42515,     0,     0,     0,     2],
        [    0,   178,   270,     0,   322, 42431,   110,     0,     4],
        [    0,     0,   114,     0,     0,     0, 37536,  4867,     0],
        [    0,     0,   135,     0,     0,     0,  3867, 39308,     0],
        [  796,     0,   414,     0,   734,    61,     0,     0, 41141]])
Epoch end: Train, epoch number: 17
TrainEpoch/CM
 tensor([[39834,     0,  1120,     0,  1301,     0,     0,     0,  1051],
        [    0, 43114,   210,     0,     0,     0,    73,     0,     0],
        [  487,     0, 41271,     0,  1425,    12,     4,     0,   184],
        [    

Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.986. Signaling Trainer to stop.


In [64]:

print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = subset.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/official_logs/multi-layer_30_from_rp4_weights_30s_Xi_sensor_1_rp3/version_0/checkpoints/epoch=12-step=455.ckpt
tensor(0.9859, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[469,   0,   3,   0,   0,   0,   0,   0,   8],
        [  0, 474,   0,   0,   0,   0,   0,   0,   0],
        [  1,   0, 478,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0, 462,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 485,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 478,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0, 465,  17,   0],
        [  0,   0,   0,   0,   0,   0,  23, 462,   0],
        [  8,   0,   0,   0,   0,   0,   0,   0, 477]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9858713150024414
      Test/F1Score          0.9861435890197754
        Test/Loss          0.038981132209300995
     Test/Precision         0

[{'Test/Loss': 0.038981132209300995,
  'Test/Accuracy': 0.9858713150024414,
  'Test/Precision': 0.9867787957191467,
  'Test/Recall': 0.9858713150024414,
  'Test/F1Score': 0.9861435890197754,
  'TestEpoch/Accuracy': 0.9862009286880493,
  'TestEpoch/Precision': 0.9862020015716553,
  'TestEpoch/Recall': 0.9862009286880493,
  'TestEpoch/F1Score': 0.9861922860145569}]

In [65]:
subset.columns.to_frame().to_csv(f"weight_selected_{TIME_WINDOW}.csv", index=False)